# TinyMS LeNet5 Server Tutorial

### In this tutorial, constructing a LeNet model, downloading dataset , training and and start the server of the model using TinyMS API will be demonstrated. 

## Prerequisite
 - Ubuntu: `18.04`
 - Python: `3.7.x`
 - Flask: `1.1.2`
 - MindSpore: `CPU-1.1.0`
 - TinyMS: `0.1.0`
 - numpy: `1.17.5`
 - opencv-python: `4.5.1.48`
 - Pillow: `8.1.0`
 - pip: `21.0.1`
 - requests: `2.18.4`
 
## Introduction

TinyMS is a high-level API which is designed for amateur of deep learning. It minimizes the number of actions of users required to construct, train, evaluate and serve a model. TinyMS also provides tutorials and documentations for developers. 

This tutorial consists of five parts, constructing the model, downloading dataset, training, define servable json and starting server, while sending a prediction request will be demonstrated in the `LeNet5_Client_tutorial.ipynb` file.

In [ ]:
import os
import json
import tinyms as ts
import tinyms.optimizers as opt
from tinyms.data import MnistDataset, download_dataset
from tinyms.data.transforms import TypeCast
from tinyms.vision import Inter, Resize, Rescale, HWC2CHW
from tinyms.model import lenet5
from tinyms.model import Model
from tinyms.serving import *
from tinyms.metrics import Accuracy
from tinyms.losses import SoftmaxCrossEntropyWithLogits

## 1. Construct the model

TinyMS encapsulates init and construct of the LeNet5 model, the line of the code is reduced and the following code has similar functions to this:

```
    import mindspore.nn as nn

    class LeNet5(nn.Cell):
        def __init__(self, num_class=10, num_channel=1):
            super(LeNet5, self).__init__()
            self.conv1 = nn.Conv2d(num_channel, 6, 5, pad_mode='valid')
            self.conv2 = nn.Conv2d(6, 16, 5, pad_mode='valid')
            self.relu = nn.ReLU()
            self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
            self.flatten = nn.Flatten()
            self.fc1 = nn.Dense(16 * 5 * 5, 120, weight_init=Normal(0.02))
            self.fc2 = nn.Dense(120, 84, weight_init=Normal(0.02))
            self.fc3 = nn.Dense(84, num_class, weight_init=Normal(0.02))

        def construct(self, x):
            x = self.max_pool2d(self.relu(self.conv1(x)))
            x = self.max_pool2d(self.relu(self.conv2(x)))
            x = self.flatten(x)
            x = self.relu(self.fc1(x))
            x = self.relu(self.fc2(x))
            x = self.fc3(x)
            return x

    net = LeNet(class_num=10)
```

In [ ]:
net = lenet5(class_num=10)
model = Model(net)

## 2. Download dataset

The MNIST dataset will be downloaded if `mnist` folder didn't exist at the root. If `mnist` folder already exists, this step will not be performed.

In [ ]:
def create_dataset(data_path, batch_size=32, repeat_size=1,
                   num_parallel_workers=1):
    """ create Mnist dataset for train or eval.
    Args:
        data_path: Data path
        batch_size: The number of data records in each group
        repeat_size: The number of replicated data records
        num_parallel_workers: The number of parallel workers
    """
    # define dataset
    mnist_ds = MnistDataset(data_path, num_parallel_workers=num_parallel_workers,
                            shuffle=True)

    # define map operations
    c_trans = [
        Resize((32, 32), interpolation=Inter.LINEAR),  # Resize images to (32, 32)
        Rescale(1 / 0.3081, -1 * 0.1307 / 0.3081),  # normalize images
        Rescale(1.0 / 255.0, 0.0),  # rescale images
        HWC2CHW(),  # change shape from (height, width, channel) to (channel, height, width) to fit network
    ]
    type_cast_op = TypeCast(ts.int32)  # change data type of label to int32 to fit network

    # apply map operations on images
    mnist_ds = mnist_ds.map(operations=type_cast_op, input_columns="label", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=c_trans, input_columns="image", num_parallel_workers=num_parallel_workers)
    # apply batch operations
    mnist_ds = mnist_ds.batch(batch_size, drop_remainder=True)
    # apply repeat operations
    mnist_ds = mnist_ds.repeat(repeat_size)

    return mnist_ds

# download the dataset
if not os.path.exists('/root/mnist'):
    ts.data.download_dataset('mnist', '/root')
    print('************Download complete*************')
else:
    print('************Dataset already exists.**************')

## 3. Train the model

The dataset for both training and evaluation will be defined here, and the parameters for training also set in this process. A trained ckpt file will be saved to `/etc/tinyms/serving/lenet5` folder for later use, meanwhile the evaluation will be performed and the `Accuracy` can be checked

In [ ]:
# define the training and evaluation dataset
mnist_path = '/root/mnist'
batch_size = 32
train_dataset = create_dataset(os.path.join(mnist_path, "train"), batch_size=batch_size)
eval_dataset = create_dataset(os.path.join(mnist_path, "test"))

# parameters for training
lr = 0.01
momentum = 0.9
epoch_size = 1
net_loss = SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
net_opt = opt.Momentum(net.trainable_params(), lr, momentum)
net_metrics={"Accuracy": Accuracy()}

model.compile(loss_fn=net_loss, optimizer=net_opt, metrics=net_metrics)
print('************************Start training*************************')
model.train(epoch_size, train_dataset)
model.save_checkpoint('/etc/tinyms/serving/lenet5/lenet5.ckpt')
print('************************Finished training*************************')

model.load_checkpoint('/etc/tinyms/serving/lenet5/lenet5.ckpt')
print('************************Start evaluation*************************')
model.eval(eval_dataset)

## 4. Define servable.json

Define the lenet5 servable json file for model name, format and number of classes for later use. 

In [ ]:
servable_json = [{'name': 'lenet5', 
                  'description': 'This servable hosts a lenet5 model predicting numbers', 
                  'model': {
                      "name": "lenet5", 
                      "format": "ckpt", 
                      "class_num": 10}}]
os.chdir("/etc/tinyms/serving")
json_data = json.dumps(servable_json, indent=4)

with open('servable.json', 'w') as json_file:
    json_file.write(json_data)

## 5. Start server

### 5.1 Introduction
TinyMS Serving is a C/S(client/server) structure. TinyMS using [Flask](https://flask.palletsprojects.com/en/1.1.x/) whichi is a micro web framework written in python as the C/S communication tool. In order to serve a model, user must start server first. If successfully started, the server will listen to POST requests from 127.0.0.1 port 5000 sent by client and handle the requests using MindSpore backend which will construct the model, run the prediction and send the result back to the client.

### 5.2 Start server

run the following code block to start the server:

In [ ]:
start()

If you can see somethin similar to this:
```
* Serving Flask app "tinyms.serving.server.server" (lazy loading)
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
```
that means you have successfully launched a server. Next, go to 'LeNet5_Client_tutorial.ipynb' to continue.

## Shutdown server

To shut down server, if using terminal, simply CTRL + C to shutdown serving, if running in Jupyter, click `Kernel` at the top and then `Shutdown`